In [ ]:
# basics
import pandas as pd
import numpy as np

# maps
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# prep
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# feature engineering
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# modelling
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor

# evaluation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

forest = '#284e13ff'

# xgb_cv_r2 = {x: np.nan for x in list(range(5))}
# xgb_cv_mse = {x: np.nan for x in list(range(5))}
# xgb_r2 = {x: np.nan for x in list(range(5))}
# xgb_mse = {x: np.nan for x in list(range(5))}
# sample_size = {x: np.nan for x in list(range(5))}


In [ ]:
areas_list = ['', '_1950', '_2910', '_4830', '_7710']

############ PICK AREA SIZE HERE ##############
area = areas_list[4]
###############################################

file_to_load = 'data/regression_sample' + area + '.csv'
index = areas_list.index(area)


In [ ]:
df = pd.read_csv(file_to_load, sep = "\t")

# Define a dictionary with the features to convert to 'category'
to_category = {'sector_main': 'category', 'country': 'category', 'start_year_first': 'category'}

# Convert the features to 'category' dtype
df = df.astype(to_category)

In [ ]:
sector_means = df[df.defo_total > 0].groupby('sector_main').around_3.describe().reset_index()[['sector_main', 'mean', 'count']]

row_total = {'sector_main': 'total', 'mean':df[df.defo_total > 0].around_3.mean(), 'count': df[df.defo_total > 0].around_3.count()}
row_total_df = pd.DataFrame([row_total])

sector_means = pd.concat([sector_means, row_total_df], ignore_index=True).sort_values('mean')

In [ ]:
fig = plt.figure(figsize=(5, 5))

colors = [forest] * len(sector_means)
colors[9] = '#c83215'

plt.barh(sector_means.sector_main, sector_means['mean'], color = colors)
plt.title('Mean asset deforestation by sector, t = (-1,1)')
plt.show()

fig.savefig('graphs/hbar_means_by_sector.png', bbox_inches='tight')

In [ ]:
sector_counts = df.groupby('sector_main').around_3.describe().reset_index()[['sector_main', 'mean', 'count']].sort_values('count')

fig = plt.figure(figsize=(5, 5))


plt.barh(sector_counts.sector_main, sector_counts['count'], color = forest)
plt.title('Number of assets, by sector')
plt.show()

fig.savefig('graphs/hbar_sectors.png', bbox_inches='tight')
df[(df.treecover2000 > 0) & (df.defo_total > 0)].sector_main.nunique()

# Simple modelling

In [ ]:
df.columns

In [ ]:
y_var = 'around_3'

df_pred = df[df[y_var].notnull()]
df_pred = df_pred[df_pred.treecover2000 > 0]
# df_pred = df_pred[df_pred.start_year_first < 2013]
df_pred = df_pred[df_pred.defo_total > 0]

aux_country = df_pred.groupby('country').uid_gem.count().reset_index().sort_values('uid_gem').rename(columns = {'uid_gem': 'country_count'})
df_pred = pd.merge(df_pred, aux_country, how = 'inner', on = 'country')

df_pred = df_pred[df_pred.country_count > 1]

plt.hist(df_pred.around_3, bins = 20)
df_pred.describe()

# df.groupby('quintile_capacity').count()

In [ ]:
X_cols = ['sector_main', 'country', 'number_units', 'start_year_first'] #, 'defo_total']
X = df_pred[X_cols]
X_strat = df_pred[['country']]
y = df_pred[y_var].apply(lambda x: np.log(x + 1))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify = X_strat)
X_train.dtypes

In [ ]:
sample_size[index] = len(X_train)
plt.hist(y_train)

# Preprocessing

In [ ]:
# Create a ColumnTransformer to handle string data
preprocessor = ColumnTransformer(
    transformers=[
        ('country', OneHotEncoder(), ['country']),
        # ('sector_sub_first', OneHotEncoder(), ['sector_sub_first']),
        ('sector_main', OneHotEncoder(), ['sector_main']),
    ],
    remainder='passthrough'
)


# Baseline model: mean

In [ ]:
mean = np.mean(y_train)

y_pred = [mean] * len(y_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
baseline_string = f"Baseline output: r2 of {round(r2, 5)}, mse of {round(mse, 6)}"
print(baseline_string)
# y_train.describe()

# Linear regression

In [ ]:
# Create the pipeline
lm = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

lm.fit(X_train, y_train)

# KNN regression

In [ ]:
# Create the pipeline
knn = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', KNeighborsRegressor())
])

knn.fit(X_train, y_train)

# XGBoost

In [ ]:
xgbm = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(enable_categorical = True))
])
xgbm.fit(X_train, y_train)


In [ ]:
# The learning rate (Greek letter eta) controls the step size at which the optimizer makes updates to the weights. 
# A smaller eta value results in slower but more accurate updates, while a larger eta value results 
# in faster but less accurate updates.

# The subsample parameter controls the fraction of observations used for each tree. A smaller subsample 
# value results in smaller and less complex models, which can help prevent overfitting.

# For any given problem, a lower log loss value means better predictions.

# You don't need xgboost.cv to find the optimal number of trees. You can also 
# run xgboost.train with "early_stopping_rounds" set. 

X_train_t = preprocessor.fit_transform(X_train, y_train)
X_test_t = preprocessor.fit_transform(X_test, y_test)

#xgb.callback.TrainingCallback()

# res.best_iteration
# is most likely to be the last one but maybe not...

# n_estimators
# num_boosting_rounds

# Tune max_depth and min_child_weight first as they will have the highest impact on the model outcome.

# could provide a DataFrame with all desired combinations...
# then keep track of the results to choose the best...

# def fit(xgbm, **kwargs):

#     data_dmatrix = xgb.DMatrix(data=X_train_t, label=y_train)

#     # other parameters include max_depth, min_child_weight...
#     # defaults are all within a good range

#     # run cv for each model...
#     # GridSearchCV could be useful...

#     params = {'objective':'reg:squarederror',
#               'eval_metric':'rmse',
#               'eta': x[0],
#               'subsample':x[1]}
#     # stratified=True ???
#     # num_boost_round can be set high, 
#     xgb_cv = xgb.cv(dtrain=data_dmatrix, 
#                     params=params, 
#                     nfold=5, 
#                     early_stopping_rounds=50, # over all folds...
#                     metrics = 'rmse', 
#                     seed=42)
#     print(xgb_cv.shape)
#     print(xgb_cv)
#     return xgb_cv[-1:].values[0]

# grid = pd.DataFrame({'eta':[0.01,0.05,0.1]*2, 
#                      'subsample':np.repeat([0.1,0.3],3),
#                      })

# grid[['train-rmse-mean','train-rmse-std',
#     'test-rmse-mean','test-rmse-std']] = grid.apply(fit, axis=1, result_type='expand')

# params = {'objective':'reg:squarederror',
#             'eval_metric':'rmse',
#             'eta': grid.iloc[4].eta,
#             'subsample': grid.iloc[4].subsample}
# regressor = XGBRegressor(**params)

def fit_for_early_stopping(boost, X_train, y_train, cv = 5, early_stopping_rounds = 50):
    params = boost.get_params()
    xgtrain = xgb.DMatrix(X_train, y_train)
    cvresult = xgb.cv(boost.get_xgb_params(), xgtrain, num_boost_round=params['n_estimators'], nfold=cv,
            metrics='rmse', early_stopping_rounds=early_stopping_rounds)
    boost.set_params(n_estimators = cvresult.shape[0])
    print(cvresult.shape[0])

xgbr = XGBRegressor(
    learning_rate = 0.1, # eta
    n_estimators = 1000,
    max_depth=7,
    #min_child_weight=1,
    gamma = 0,
    #subsample=0.8,
    #colsample_bytree=0.8,
    #objective= 'reg:squarederror',
    seed = 42, 
    enable_categorical = True)

# fit n_estimators on training data for early stopping
fit_for_early_stopping(xgbr, X_train_t, y_train, cv = 5, early_stopping_rounds = 50)

param_grid = {
    'eta': [0.00001, 0.0001, 0.001, 0.01], #0.0001, 0.001,
    'subsample': [0.1, 0.2, 0.3, 0.4, 0.5], 
    'min_child_weight': [0.001, 0.01, 0.05, 0.1, 0.5],
    'max_depth': list(range(5, 10, 2)),
    'gamma': [0],
    'alpha': [0, 1, 10, 100],
}

gridsearch = GridSearchCV(xgbr, param_grid, verbose = 0, cv = 5, n_jobs = -1)

gridsearch.fit(X_train_t, y_train)

xgbm2 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', gridsearch.best_estimator_)
])
xgbm2.fit(X_train, y_train)

gridsearch.best_params_



# Evaluate 

In [ ]:

y_pred = lm.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Linreg output: r2 of {round(r2, 3)}, mse of {round(mse, 6)}")

# # ---------------------------------------------------------
# print("----" * 10)

# y_pred = knn.predict(X_test)

# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# print(f"KNN output: r2 of {round(r2, 3)}, mse of {round(mse, 6)}")

# # ---------------------------------------------------------
# print("----" * 10)

print("XGB:")

y_pred = xgbm.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"XGB output: r2 of {round(r2, 3)}, mse of {round(mse, 6)}")

xgb_r2[index] = round(r2, 5)
xgb_mse[index] = round(mse, 5)


print("--- on train ---")

y_pred = xgbm.predict(X_train)
mse = mean_squared_error(y_train, y_pred)   
r2 = r2_score(y_train, y_pred)
print(f"r2 of {round(r2, 3)}, mse of {round(mse, 6)}")

print("XGB (cv):")

y_pred = xgbm2.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"XGB (cv) output: r2 of {round(r2, 3)}, mse of {round(mse, 6)}")

xgb_cv_r2[index] = round(r2, 5)
xgb_cv_mse[index] = round(mse, 5) 


print("--- on train ---")

y_pred = xgbm2.predict(X_train)
mse = mean_squared_error(y_train, y_pred)   
r2 = r2_score(y_train, y_pred)
print(f"r2 of {round(r2, 3)}, mse of {round(mse, 6)}")

In [ ]:
print(f"XGB CV MSE: {xgb_cv_mse}")
print(f"XGB CV R2: {xgb_cv_r2}")
print("------" * 10)
print(f"XGB MSE: {xgb_mse}")
print(f"XGB R2: {xgb_r2}")
print("------" * 10)
print(f"Sample size: {sample_size}")
print(baseline_string)